# Converting developed model to Vetiver endpoint

In [1]:
from vetiver.server import predict, vetiver_endpoint

In [2]:
import pins

In [3]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [4]:
import os

In [5]:
board = pins.board_rsconnect(server_url="https://colorado.rstudio.com/rsc",
                            allow_pickle_read=True)

In [11]:
# read the pinned model
model = board.pin_read("gagan/bikeshare-rf-2306")
model

RandomForestRegressor(max_depth=20, max_features='sqrt', min_samples_leaf=0.5,
                      min_samples_split=0.5, n_jobs=6, random_state=0,
                      verbose=100)

In [6]:
X_test = board.pin_read("gagan/bike_x_test")
X_test

,hour,month,lat,lon,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
190,4,4,38.933668,-76.991016,0,0,0,0,0,1,0
191,4,4,38.933668,-76.991016,0,0,0,0,0,0,1
192,4,4,38.933668,-76.991016,0,0,0,0,1,0,0
193,4,4,38.933668,-76.991016,1,0,0,0,0,0,0
194,4,4,38.933668,-76.991016,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
12259907,3,4,38.933668,-76.991016,0,0,1,0,0,0,0
12259908,3,5,38.933668,-76.991016,0,0,0,1,0,0,0
12259909,3,5,38.933668,-76.991016,0,1,0,0,0,0,0
12259910,3,5,38.933668,-76.991016,0,0,0,0,0,1,0


In [12]:
X_train = board.pin_read("gagan/bike_x_train")
X_train

,hour,month,lat,lon,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,4,10,38.933668,-76.991016,0,0,0,1,0,0,0
1,4,10,38.933668,-76.991016,0,1,0,0,0,0,0
2,4,10,38.933668,-76.991016,0,0,0,0,0,1,0
3,4,10,38.933668,-76.991016,0,0,0,0,0,0,1
4,4,10,38.933668,-76.991016,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2035446,5,8,38.882788,-77.103148,1,0,0,0,0,0,0
2035447,5,8,38.882788,-77.103148,0,0,1,0,0,0,0
2035448,5,8,38.882788,-77.103148,0,0,0,1,0,0,0
2035449,5,8,38.882788,-77.103148,0,1,0,0,0,0,0


In [13]:
from vetiver import VetiverModel, VetiverAPI, vetiver_endpoint,vetiver_write_app,pin_read_write

In [14]:
v = VetiverModel(model, "gagan/bikeshare-rf-2306", save_ptype = True, ptype_data=X_train)

In [15]:
v

In [16]:
app = VetiverAPI(v, check_ptype=True)
app

In [17]:
pin_read_write.vetiver_pin_write(
board,
v
)


Writing pin:
Name: 'gagan/bikeshare-rf-2306'
Version: 20220623T172924Z-282d5


In [22]:
# create fastAPI endpoint
# (will be replaced by Connect deplpoyment function)
vetiver_write_app(
board,
"gagan/bikeshare-rf-2306",
file="model_new.py"
)

/usr/home/gagan/bike_predict_vetiver/.bike_predict_vetiver/lib/python3.9/site-packages/vetiver/write_fastapi.py:20: UserWarning: Pinned vetiver model has no active version and no datetime on versions,
              Do you need to check your pinned model?
              Using version 58019
  warnings.warn(


In [23]:
from rsconnect.api import RSConnectServer
from rsconnect.actions import deploy_python_fastapi

In [24]:
rsc_server=os.getenv("CONNECT_SERVER")

In [25]:
rsc_key=os.getenv("CONNECT_API_KEY")

In [26]:
connect_server = RSConnectServer(url=rsc_server, api_key=rsc_key)

In [27]:
deploy_python_fastapi(
    connect_server = connect_server,
    title = "Random Forest model for bike share 23-06",
    directory = ".",
    python = ".bike_predict_vetiver/bin/python",
    extra_files = None,
    entry_point = "model_new:api",
    excludes = None,
)

('https://colorado.rstudio.com/rsc/connect/#/apps/c6adc932-88f0-4536-a5c1-f6c1f6086244',
 ['Building FastAPI application...',
  'Bundle created with Python version 3.9.6 is compatible with environment Kubernetes::ghcr.io/rstudio/content-pro:r4.0.5-py3.9.2-bionic with Python version 3.9.2 from /opt/python/3.9.2/bin/python3.9 ',
  'Bundle requested Python version 3.9.6; using /opt/python/3.9.2/bin/python3.9 from Kubernetes::ghcr.io/rstudio/content-pro:r4.0.5-py3.9.2-bionic which has version 3.9.2',
  'Determining session server location ...',
  'Connecting to session server http://172.19.57.196:32636 ...',
  '2022/06/23 17:41:42.477249279 Running on host: python-environment-restore-spz57-tgdwh',
  '2022/06/23 17:41:42.477266021 Working directory: /opt/rstudio-connect/mnt/app',
  '2022/06/23 17:41:42.477342326 Environment will be built with Python "3.9.2 (default, Mar  3 2021, 20:02:32)  [GCC 7.3.0]" at /opt/python/3.9.2/bin/python3.9',
  '2022/06/23 17:41:42.477742162 Running as user: 99